In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from main import TradingBot
import random
from IPython.display import clear_output

tb = TradingBot()

In [11]:
tb.loadData(timePeriod='2y', label='NIFTY 50', forceUpdate=True)
tb.computeSignal()

100%|██████████| 52/52 [00:56<00:00,  1.08s/it]


In [32]:
for stock in tb.data:
    indicator = tb.indicatorCollection[stock]["sri"]
    pattern = indicator.df[indicator.df['candlestick_pattern'] == 'CDL3BLACKCROWS_Bear']
    if len(pattern.index)>0:
        print(stock)

BRITANNIA


In [33]:
indicator = tb.indicatorCollection['BRITANNIA']["sri"]
indicator.df[indicator.df['candlestick_pattern'] == 'CDL3BLACKCROWS_Bear']

,Date,Open,High,Low,Close,Volume,RSI,candlestick_rank,candlestick_pattern,candlestick_match_count,EMA100,Level,Signal,SignalMarker,Target,Stoploss
95,2021-12-07,3502.05,3529.0,3467.0,3474.2,254445,35.032013,3.0,CDL3BLACKCROWS_Bear,1.0,NaN,0,0,NaN,NaN,NaN


In [34]:
indicator.showIndicator(100)